In [8]:
from scrape import get_results
from scrape import get_data
from scrape import grab_years_played
from scrape import revised_run
from scrape import train_test

2022 WR Projections

In [11]:
X_train2022, X_test2022 = train_test('wr', num_years=5, year_for=2022)

In [12]:
results = revised_run(X_train2022, X_test2022, 'wr', save_csv=True, year_for='2022')
results.head(50)

======= Fold 0 ========
Our accuracy on the validation set is 3.401458111898495
======= Fold 1 ========
Our accuracy on the validation set is 3.6087294375497905
======= Fold 2 ========
Our accuracy on the validation set is 4.132186289871178
======= Fold 3 ========
Our accuracy on the validation set is 4.411943458624885
======= Fold 4 ========
Our accuracy on the validation set is 3.956806910263533
iteration 1: Dimensions = (91, 2)
iteration 2: Dimensions = (182, 2)
iteration 3: Dimensions = (273, 2)
iteration 4: Dimensions = (364, 2)
iteration 5: Dimensions = (455, 2)


,name,proj fpts,class,recent rank
0,justin-jefferson,18.430186,0,1
1,aj-brown,17.814517,0,6
2,ceedee-lamb,17.101213,0,5
3,jamarr-chase,17.090376,0,11
4,amonra-stbrown,16.956436,1,7
5,davante-adams,16.845435,0,3
6,stefon-diggs,16.668903,0,4
7,tyreek-hill,16.564748,0,2
8,jaylen-waddle,16.413802,0,8
9,cooper-kupp,16.118074,1,23


In [14]:
X_train2022, X_test2022 = train_test('wr')
results = revised_run(X_train2022, X_test2022, 'wr', model_ = 'svr')
results.head(50)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
======= Fold 0 ========
Our accuracy on the validation set is 3.5521996924502433
Fitting 5 folds for each of 25 candidates, totalling 125 fits
======= Fold 1 ========
Our accuracy on the validation set is 3.6868357437989334
Fitting 5 folds for each of 25 candidates, totalling 125 fits
======= Fold 2 ========
Our accuracy on the validation set is 3.9976022439083305
Fitting 5 folds for each of 25 candidates, totalling 125 fits
======= Fold 3 ========
Our accuracy on the validation set is 4.316232267972919
Fitting 5 folds for each of 25 candidates, totalling 125 fits
======= Fold 4 ========
Our accuracy on the validation set is 3.9396228104120605
Fitting 5 folds for each of 25 candidates, totalling 125 fits
iteration 1: Dimensions = (91, 2)
Fitting 5 folds for each of 25 candidates, totalling 125 fits
iteration 2: Dimensions = (182, 2)
Fitting 5 folds for each of 25 candidates, totalling 125 fits
iteration 3: Dimensions = (273,

,name,proj fpts,class,recent rank
0,justin-jefferson,18.165152,0,1
1,davante-adams,17.786420,0,3
2,aj-brown,17.364404,0,6
3,tyreek-hill,17.224564,0,2
4,ceedee-lamb,16.885012,0,5
5,stefon-diggs,16.728406,0,4
6,jamarr-chase,16.705152,0,11
7,cooper-kupp,16.681541,1,23
8,amonra-stbrown,16.195248,1,7
9,jaylen-waddle,15.352503,0,8
